# Titanic Survival Prediction

Dataset Description :   

<br>PassengerId: and id given to each traveler on the boat
<br>Pclass: the passenger class. It has three possible values: 1,2,3 (first, second and third class)
<br>The Name of the passeger
<br>The Sex
<br>The Age
<br>SibSp: number of siblings and spouses traveling with the passenger
<br>Parch: number of parents and children traveling with the passenger
<br>The ticket number
<br>The ticket Fare
<br>The cabin number
<br>The embarkation. This describes three possible ports at which people boarded Titanic. Three possible values S,C,Q

In [211]:
train_location = 'train.csv'
Original_train_location = 'trainOriginal.csv'
test_location = 'test.csv'
Original_test_location = 'testOriginal.csv'

In [212]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.grid_search import RandomizedSearchCV
import matplotlib as plt

WOA - Without Age 
<br> WA - With Age

In [213]:
trainData = pd.read_csv(train_location)
testData = pd.read_csv(test_location)

In [214]:
trainData.head()

,Name,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,"Braund, Mr. Owen Harris",3,0,22.0,1,0,7.2500,1.0,0
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,1,38.0,1,0,71.2833,0.0,1
2,"Heikkinen, Miss. Laina",3,1,26.0,0,0,7.9250,1.0,1
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,1,35.0,1,0,53.1000,1.0,1
4,"Allen, Mr. William Henry",3,0,35.0,0,0,8.0500,1.0,0


In [215]:
trainData.shape

(891, 9)

In [216]:
testData.head()

,Name,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,"Kelly, Mr. James",3,0,34.5,0,0,7.8292,2
1,"Wilkes, Mrs. James (Ellen Needs)",3,1,47.0,1,0,7.0000,1
2,"Myles, Mr. Thomas Francis",2,0,62.0,0,0,9.6875,2
3,"Wirz, Mr. Albert",3,0,27.0,0,0,8.6625,1
4,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",3,1,22.0,1,1,12.2875,1


In [217]:
testData.shape

(418, 8)

## Replacing Name with title 

Adding column titles:
    <br> 0 - Mr.
    <br> 1 - Master.
    <br> 2 - Mrs.
    <br> 3 - Miss.
    

In [218]:
titles = []
k=0
for i in trainData.iloc[:,0]:
    foundTitle = 0
    for title in i.split():
        if (title in ['Mr.', 'Don.', 'Dr.', 'Major.', 'Col.', 'Rev.', 'Sir.', 'Capt.']):
            titles.append(0)
            foundTitle = 1
            break
        elif (title == 'Master.'):
            titles.append(1)
            foundTitle = 1
            break
        elif (title in ['Mrs.', 'Mme.', 'Lady.', 'Countess.']):
            titles.append(2)
            foundTitle = 1
            break
        elif (title in ['Miss.', 'Ms.', 'Mlle.']):
            titles.append(3)
            foundTitle = 1
            break
    
    if(foundTitle == 0):
        titles.append(0)
        print(i)

Reuchlin, Jonkheer. John George


In [219]:
len(titles)

891

In [220]:
trainData.insert(loc=1, column='Title', value=titles)
trainData = trainData.drop(labels=['Name'], axis=1) 
trainData.head()

,Title,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,0,3,0,22.0,1,0,7.2500,1.0,0
1,2,1,1,38.0,1,0,71.2833,0.0,1
2,3,3,1,26.0,0,0,7.9250,1.0,1
3,2,1,1,35.0,1,0,53.1000,1.0,1
4,0,3,0,35.0,0,0,8.0500,1.0,0


In [221]:
titles = []
k=0
for i in testData.iloc[:,0]:
    foundTitle = 0
    for title in i.split():
        if (title in ['Mr.', 'Don.', 'Dr.', 'Major.', 'Col.', 'Rev.', 'Sir.', 'Capt.']):
            titles.append(0)
            foundTitle = 1
            break
        elif (title == 'Master.'):
            titles.append(1)
            foundTitle = 1
            break
        elif (title in ['Mrs.', 'Mme.', 'Lady.', 'Countess.']):
            titles.append(2)
            foundTitle = 1
            break
        elif (title in ['Miss.', 'Ms.', 'Mlle.']):
            titles.append(3)
            foundTitle = 1
            break
    
    if(foundTitle == 0):
        titles.append(3)
        print(i)

Oliva y Ocana, Dona. Fermina


In [222]:
len(titles)

418

In [223]:
testData.insert(loc=1, column='Title', value=titles)
testData = testData.drop(labels=['Name'], axis=1) 
testData.head()

,Title,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,34.5,0,0,7.8292,2
1,2,3,1,47.0,1,0,7.0000,1
2,0,2,0,62.0,0,0,9.6875,2
3,0,3,0,27.0,0,0,8.6625,1
4,2,3,1,22.0,1,1,12.2875,1


## Loading train and test data 
<br> With age - (predicting age with help Title and Fare) 
<br> Without age (dropping those rows where age is missing)

In [224]:
trainData.shape

(891, 9)

In [225]:
X_train_WA = trainData.iloc[:,:8].values
Y_train_WA = trainData.iloc[:,8].values

In [226]:
# dropping any row where age is nan because it is a huge factor
trainData_WOA = trainData.dropna(how='any',axis=0) 
X_train_WOA = trainData_WOA.iloc[:,:7].values
Y_train_WOA = trainData_WOA.iloc[:,7].values

In [227]:
testData.shape

(418, 8)

In [228]:
testOriginalData = pd.read_csv(Original_test_location)

In [229]:
X_test_WA = testData.iloc[:,:8].values

In [230]:
testData_WOA = testData.dropna(how='any',axis=0) 
X_test_WOA = testData.iloc[:,:8].values

## Replacing missing Age with the mean of respective Titles (Mr/Master/Mrs/Miss)

In [241]:
ageMr = 0
countMr = 0
ageMaster = 0
countMaster = 0
ageMrs = 0
countMrs = 0
ageMiss = 0
countMiss = 0
missing = 0

for index, row in trainData.iterrows():
    if(row.isnull().any()):
        missing = missing + 1
    else:
        if(row['Title'] == 0):
            ageMr = ageMr + row['Age']
            countMr = countMr + 1 
        elif(row['Title'] == 1):
            ageMaster = ageMaster + row['Age']
            countMaster = countMaster + 1
        elif(row['Title'] == 2):
            ageMrs = ageMrs + row['Age']
            countMrs = countMrs + 1
        elif(row['Title'] == 3):
            ageMiss = ageMiss + row['Age']
            countMiss = countMiss + 1   

In [242]:
# Counting number of Mr, Master, Mrs, Miss, Missing Age in training set
print(countMr, countMaster, countMrs, countMiss, missing)

538 40 127 184 2


In [233]:
# Their average Ages
print(ageMr/countMr, ageMaster/countMaster, ageMrs/countMrs, ageMiss/countMiss)

33.02272727272727 4.574166666666667 35.63636363636363 21.736486486486488


In [234]:
# Replacing Missing Age values
for index, row in trainData.iterrows():
    if(row.isnull().any()):
        if(row['Title'] == 0):
            trainData.set_value(index,'Age',ageMr/countMr)
        elif(row['Title'] == 1):
            trainData.set_value(index,'Age',ageMaster/countMaster)
        elif(row['Title'] == 2):
            trainData.set_value(index,'Age',ageMrs/countMrs)
        elif(row['Title'] == 3):
            trainData.set_value(index,'Age',ageMiss/countMiss)

In [247]:
trainData.head(10)

,Title,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,0,3,0,22.000000,1,0,7.2500,1.0,0
1,2,1,1,38.000000,1,0,71.2833,0.0,1
2,3,3,1,26.000000,0,0,7.9250,1.0,1
3,2,1,1,35.000000,1,0,53.1000,1.0,1
4,0,3,0,35.000000,0,0,8.0500,1.0,0
5,0,3,0,33.022727,0,0,8.4583,2.0,0
6,0,1,0,54.000000,0,0,51.8625,1.0,0
7,1,3,0,2.000000,3,1,21.0750,1.0,0
8,2,3,1,27.000000,0,2,11.1333,1.0,1
9,2,2,1,14.000000,1,0,30.0708,0.0,1


In [248]:
# Replacing the 2 missing embarked values with 0 as frequency of embarked = 0 is highest
for index, row in trainData.iterrows():
    if(row.isnull().any()):
        trainData.set_value(index,'Embarked',0)

In [249]:
for index, row in trainData.iterrows():
    if(row.isnull().any()):
        print('Missing value found')

For Test set

In [268]:
for index, row in testData.iterrows():
    if(row.isnull().any()):
        if(row['Title'] == 0):
            testData.set_value(index,'Age',ageMr/countMr)
        elif(row['Title'] == 1):
            testData.set_value(index,'Age',ageMaster/countMaster)
        elif(row['Title'] == 2):
            testData.set_value(index,'Age',ageMrs/countMrs)
        elif(row['Title'] == 3):
            testData.set_value(index,'Age',ageMiss/countMiss)

In [270]:
for index, row in trainData.iterrows():
    if(row.isnull().any()):
        print('Missing value found')

All missing values have been dealt with

In [271]:
X_train_WA = trainData.iloc[:,:8].values
Y_train_WA = trainData.iloc[:,8].values
X_test_WA = testData.iloc[:,:8].values

## Building Models

In [276]:
X_train_WA.shape

(891, 8)

In [277]:
Y_train_WA.shape

(891,)

In [278]:
X_test_WA.shape

(418, 8)

In [254]:
def float_range(start,stop,step):
    l = []
    l.append(start);
    
    while(start+step<=stop):
        start = start + step
        l.append(start)
        
    return l;

In [255]:
import numpy as np

In [256]:
np.linspace(0.01,0.3,8)

array([0.01      , 0.05142857, 0.09285714, 0.13428571, 0.17571429,
       0.21714286, 0.25857143, 0.3       ])

In [257]:
l = np.floor(np.linspace(0,1000,30))

In [258]:
l

array([   0.,   34.,   68.,  103.,  137.,  172.,  206.,  241.,  275.,
        310.,  344.,  379.,  413.,  448.,  482.,  517.,  551.,  586.,
        620.,  655.,  689.,  724.,  758.,  793.,  827.,  862.,  896.,
        931.,  965., 1000.])

In [259]:
range(1,6,2)

range(1, 6, 2)

# XGBoost

In [260]:
XGB_param_grid = {
                    'max_depth':[3,4,5,6,7,8,9,10],
                    'min_child_weight':[1,3,5,7,9],
                    'learning_rate':[0.001,0.01,0.05,0.1,0.15,0.21,0.25],
                    'n_estimators':[10,30,60,100,200,300,500,700,1000],
                    'gamma':[i/10.0 for i in range(0,5)],
                    'subsample':[i/100.0 for i in range(75,90,5)],
                    'colsample_bytree':[i/100.0 for i in range(75,90,5)],
                    'reg_alpha':[1e-5, 1e-2, 0, 0.001, 0.005, 0.05, 0.1, 1, 100 ]                   
                 }

### Init signature:  

XGBClassifier(max_depth=3, learning_rate=0.1, n_estimators=100, silent=True, objective='binary:logistic', booster='gbtree', n_jobs=1, nthread=None, gamma=0, min_child_weight=1, max_delta_step=0, subsample=1, colsample_bytree=1, colsample_bylevel=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, base_score=0.5, random_state=0, seed=None, missing=None, **kwargs)

In [261]:
from xgboost.sklearn import XGBClassifier

In [262]:
from sklearn.grid_search import RandomizedSearchCV

In [263]:
xgb = XGBClassifier()

In [264]:
Y_train_WA.shape

(891,)

In [265]:
%%time 
randcvXGB = RandomizedSearchCV(xgb, XGB_param_grid, cv=5, n_iter=100, scoring='accuracy',verbose=2)
randcvXGB.fit(X_train_WA, Y_train_WA)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] subsample=0.75, reg_alpha=0.1, n_estimators=300, min_child_weight=3, max_depth=7, learning_rate=0.001, gamma=0.4, colsample_bytree=0.75 


C:\Users\DELL PC\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
C:\Users\DELL PC\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[CV]  subsample=0.75, reg_alpha=0.1, n_estimators=300, min_child_weight=3, max_depth=7, learning_rate=0.001, gamma=0.4, colsample_bytree=0.75 -   0.3s
[CV] subsample=0.75, reg_alpha=0.1, n_estimators=300, min_child_weight=3, max_depth=7, learning_rate=0.001, gamma=0.4, colsample_bytree=0.75 
[CV]  subsample=0.75, reg_alpha=0.1, n_estimators=300, min_child_weight=3, max_depth=7, learning_rate=0.001, gamma=0.4, colsample_bytree=0.75 -   0.1s
[CV] subsample=0.75, reg_alpha=0.1, n_estimators=300, min_child_weight=3, max_depth=7, learning_rate=0.001, gamma=0.4, colsample_bytree=0.75 


C:\Users\DELL PC\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\DELL PC\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[CV]  subsample=0.75, reg_alpha=0.1, n_estimators=300, min_child_weight=3, max_depth=7, learning_rate=0.001, gamma=0.4, colsample_bytree=0.75 -   0.1s
[CV] subsample=0.75, reg_alpha=0.1, n_estimators=300, min_child_weight=3, max_depth=7, learning_rate=0.001, gamma=0.4, colsample_bytree=0.75 
[CV]  subsample=0.75, reg_alpha=0.1, n_estimators=300, min_child_weight=3, max_depth=7, learning_rate=0.001, gamma=0.4, colsample_bytree=0.75 -   0.1s
[CV] subsample=0.75, reg_alpha=0.1, n_estimators=300, min_child_weight=3, max_depth=7, learning_rate=0.001, gamma=0.4, colsample_bytree=0.75 


...

C:\Users\DELL PC\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[CV]  subsample=0.75, reg_alpha=0.001, n_estimators=1000, min_child_weight=9, max_depth=8, learning_rate=0.25, gamma=0.1, colsample_bytree=0.75 -   0.3s
[CV] subsample=0.75, reg_alpha=0.001, n_estimators=1000, min_child_weight=9, max_depth=8, learning_rate=0.25, gamma=0.1, colsample_bytree=0.75 


C:\Users\DELL PC\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[CV]  subsample=0.75, reg_alpha=0.001, n_estimators=1000, min_child_weight=9, max_depth=8, learning_rate=0.25, gamma=0.1, colsample_bytree=0.75 -   0.5s
[CV] subsample=0.75, reg_alpha=0.001, n_estimators=1000, min_child_weight=9, max_depth=8, learning_rate=0.25, gamma=0.1, colsample_bytree=0.75 
[CV]  subsample=0.75, reg_alpha=0.001, n_estimators=1000, min_child_weight=9, max_depth=8, learning_rate=0.25, gamma=0.1, colsample_bytree=0.75 -   0.5s
Wall time: 1min 18s


C:\Users\DELL PC\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  1.3min finished


In [266]:
print(randcvXGB.best_score_)
print(randcvXGB.best_params_)

0.8473625140291807
{'subsample': 0.8, 'reg_alpha': 0.01, 'n_estimators': 200, 'min_child_weight': 9, 'max_depth': 8, 'learning_rate': 0.21, 'gamma': 0.2, 'colsample_bytree': 0.85}


In [279]:
tunedXGB= XGBClassifier(subsample=0.8, reg_alpha= 0.01 , min_child_weight= 9, learning_rate= 0.21, gamma= 0.2, colsample_bytree= 0.85, n_estimators=200, max_depth=8)

In [280]:
%time
tunedXGB.fit(X_train_WA, Y_train_WA)

Wall time: 0 ns


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.85, gamma=0.2, learning_rate=0.21,
       max_delta_step=0, max_depth=8, min_child_weight=9, missing=None,
       n_estimators=200, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0.01,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.8)

In [281]:
predictions = tunedXGB.predict(X_test_WA)

C:\Users\DELL PC\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [286]:
len(predictions)

418

In [287]:
X_test_WA.shape

(418, 8)

In [288]:
testOriginalData.shape

(418, 11)

In [289]:
results = pd.DataFrame({
    "PassengerId": testOriginalData['PassengerId'],
    "Survived": predictions})

In [290]:
results.to_csv('titanic_XGB_withReplacedAge.csv', index=False)   